In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import jax
import jax.numpy as jnp
from jax import random


ModuleNotFoundError: jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.

In [3]:
class Projector(nn.Module):
    """
    Making projection matrix(Q, K, V) for each attention head
    When you call this class, it returns projection matrix of each attention head
    For example, if you call this class with 8 heads, it returns 8 set of projection matrices (Q, K, V)
    Args:
        num_heads: number of heads in MHA, default 8
        dim_head: dimension of each attention head, default 64
    """
    def __init__(self, num_heads: int = 8, dim_head: int = 64) -> None:
        super(Projector, self).__init__()
        self.dim_model = num_heads * dim_head
        self.num_heads = num_heads
        self.dim_head = dim_head

    def __call__(self):
        fc_q = nn.Linear(self.dim_model, self.dim_head)
        fc_k = nn.Linear(self.dim_model, self.dim_head)
        fc_v = nn.Linear(self.dim_model, self.dim_head)
        return fc_q, fc_k, fc_v


class MultiHeadAttention(nn.Module):
    """
    Class for multi-head attention (MHA) module in vanilla transformer
    We apply linear transformation to input vector by each attention head's projection matrix (8, 512, 64)
    Other approaches are possible, such as using one projection matrix for all attention heads (1, 512, 512)
    and then split into each attention heads (8. 512, 64)
    Args:
        dim_model: dimension of model's latent vector space, default 512 from official paper
        num_heads: number of heads in MHA, default 8 from official paper
        dropout: dropout rate, default 0.1
    Math:
        MHA(Q, K, V) = Concat(Head1, Head2, ... Head8) * W_concat
    Reference:
        https://arxiv.org/abs/1706.03762
    """
    def __init__(self, dim_model: int = 512, num_heads: int = 8, dropout: float = 0.1) -> None:
        super(MultiHeadAttention, self).__init__()
        self.dim = dim_model
        self.num_heads = num_heads
        self.dropout = dropout
        self.dim_head = int(self.dim / self.num_heads)  # dimension of each attention head
        self.dot_scale = torch.sqrt(torch.tensor(self.dim_head))  # scale factor for Q•K^T Result

        # linear combination: projection matrix(Q_1, K_1, V_1, ... Q_n, K_n, V_n) for each attention head
        self.projector = Projector(self.num_heads, self.dim_head)  # init instance
        self.projector_list = [list(self.projector()) for _ in range(self.num_heads)]  # call instance
        self.fc_concat = nn.Linear(self.dim, self.dim)  # for concatenation of each attention head

    def forward(self, x: torch.Tensor, mask: bool = None) -> torch.Tensor:
        """
        1) make Q, K, V matrix for each attention head: [BS, HEAD, SEQ_LEN, DIM_HEAD], ex) [10, 8, 512, 64]
        2) Do self-attention in each attention head
            - Matmul (Q, K^T) with scale factor (sqrt(DIM_HEAD))
            - Mask for padding token (Option for Decoder)
            - Softmax
            - Matmul (Softmax, V)
        3) Concatenate each attention head & linear transformation (512, 512)
        """
        # 1) make Q, K, V matrix for each attention head
        Q, K, V = [], [], []

        for i in range(self.num_heads):
            Q.append(self.projector_list[i][0](x))
            K.append(self.projector_list[i][1](x))
            V.append(self.projector_list[i][2](x))

        Q = torch.stack(Q, dim=1)
        K = torch.stack(K, dim=1)
        V = torch.stack(V, dim=1)
        # 2) Do self-attention in each attention head
        attention_score = torch.matmul(Q, K.transpose(-1, -2)) / self.dot_scale
        if mask is not None:  # for padding token
            attention_score[mask] = float('-inf')
        attention_dist = F.softmax(attention_score, dim=-1)  # [BS, HEAD, SEQ_LEN, SEQ_LEN]
        attention_matrix = torch.matmul(attention_dist, V).transpose(1, 2).reshape(x.shape[0], x.shape[1], self.dim)  # [BS, SEQ_LEN, DIM]

        # 3) Concatenate each attention head & linear transformation (512, 512)
        x = self.fc_concat(attention_matrix)
        return x

In [ ]:
""" Debug for MultiHeadAttention """

x = torch.randn(10, 512, 512)
test_head = MultiHeadAttention()
test_result = test_head(x)
test_result, test_result.shape

In [47]:
""" torch.reshape test for making input shape in Vision Transformers """
patch_size, num_patches = 16, 32
x = torch.randn(10, 3, 512, 512)
x = x.reshape(x.shape[0], num_patches**2, patch_size**2 * x.shape[1])
x.shape

torch.Size([10, 1024, 768])

In [48]:
""" Check Input Embedding shape """
input_embedding = nn.Linear(768, 1024)
x = input_embedding(x)
x.shape

torch.Size([10, 1024, 1024])

In [51]:
""" make classification token for Vision Transformers """
cls_token = torch.zeros(x.shape[0], 1, x.shape[2])  # can change init method
cls_token.shape

torch.Size([10, 1, 1024])

In [52]:
torch.cat([cls_token, x], dim=1).shape

torch.Size([10, 1025, 1024])

In [11]:
""" Test for Hybrid Model """
x = torch.randn(10, 3, 512, 512)
dim_model = 512
patch_size = 16
num_patches = 32
conv = nn.Conv2d(
            in_channels=3,
            out_channels=dim_model,
            kernel_size=patch_size,
            stride=16
)
x = conv(x).reshape(x.shape[0], dim_model, num_patches**2).transpose(-1, -2)
x.shape

torch.Size([10, 1024, 512])

In [4]:
32*32

1024

In [20]:
""" Test for DeBERTa Disentangled Self-Attention """
batch, sequence, dim_model, dim_head, k = 10, 512, 2048, 64, 512
x = torch.randn(sequence, dim_model)  # [Batch, Sequence, Dim]
p_x = torch.randn(2*k, dim_model)
print(x.shape)
print(p_x.shape)

fc_q = nn.Linear(dim_model, dim_head)
fc_k = nn.Linear(dim_model, dim_head)
fc_v = nn.Linear(dim_model, dim_head)
fc_qr = nn.Linear(dim_model, dim_head)  # projector for Relative Position Query matrix
fc_kr = nn.Linear(dim_model, dim_head)  # projector for Relative Position Key matrix

q = fc_q(x)
kr = fc_kr(p_x)
print(q.shape)
print(kr.shape)

tmp_c2p= torch.stack(
    [torch.matmul(q[i, :], kr.transpose(-1, -2)) for i in range(x.shape[0])],
    dim=0
)

tmp_c2p.shape

torch.Size([512, 2048])
torch.Size([1024, 2048])
torch.Size([512, 64])
torch.Size([1024, 64])


torch.Size([512, 1024])

torch.Size([512, 1024])